# PHONOS
## Analyze source collection, plot and cluster

This notebook includes the code to analyze the collection of sounds compiled in the previous notebook and that will be later used as the source collection in our audio mosaicing code. The notebook also contains the code to analyze the target audio file that will be later reconstructed using sound chunks from the source collection.

The audio analysis carried out in this notebook uses the Pythonn bindings of the Essentia library which was introduced in the first session of AMPLAB. Please make sure you checked the [Essentia Python tutorial](https://essentia.upf.edu/documentation/essentia_python_tutorial.html) to get familiarized with using Essentia in Python. Also useful is to always have a browser tab opened with Essentia's [Algorithms Reference](https://essentia.upf.edu/documentation/algorithms_reference.html) documentation page.

In [1]:
!pip3 install librosa

    100% |████████████████████████████████| 1.6MB 350kB/s ta 0:00:011   11% |███▉                            | 184kB 2.2MB/s eta 0:00:01
    100% |████████████████████████████████| 286kB 2.3MB/s ta 0:00:011
    100% |████████████████████████████████| 327kB 3.2MB/s eta 0:00:01
    100% |████████████████████████████████| 3.3MB 211kB/s eta 0:00:01    27% |████████▋                       | 901kB 2.3MB/s eta 0:00:02    60% |███████████████████▌            | 2.0MB 5.0MB/s eta 0:00:01
    100% |████████████████████████████████| 16.8MB 75kB/s  eta 0:00:01  1% |▋                               | 327kB 2.5MB/s eta 0:00:07    56% |██████████████████              | 9.5MB 6.2MB/s eta 0:00:02    63% |████████████████████▏           | 10.6MB 6.2MB/s eta 0:00:02
  Running setup.py bdist_wheel for librosa ... done
  Stored in directory: /root/.cache/pip/wheels/ce/49/68/87ba660f30e3977f0778e39ee2e944629cd37c2a0ce41f9ff1
  Running setup.py bdist_wheel for audioread ... done
  Stored in directory: /root/.c

In [91]:
import os 
from os import listdir
from os.path import isfile, join
import codecs, json 
import pandas as pd
import essentia
import essentia.standard as estd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Audio

### build sound collection

In [75]:
DATASET_LOCAL_DIR = './phonos_dataset_local/' # notebook can't use absolute paths...
DATASET_ABSOLUTE_DIR = '/Users/lluissuros/Documents/Datasets/phonos_dataset/' # ... but SC needs them
DATAFRAME_CSV_TRACKS = './files/tracks.csv'

def is_sound(file):
    return file.lower().endswith(('.wav', '.aiff'))

def make_pandas_record(sound_name, file_path = DATASET_DIR): 
    """Create a dictionary with the metadata that we want to store for each sound.""" 
    record = {}
#    record = {key: sound_name.as_dict()[key] for key in METADATA_FIELDS}
    record['track_id'] = sound_name # name will be id
    record['path'] = file_path + sound_name
    record['absolute_path'] = DATASET_ABSOLUTE_DIR + sound_name
    return record

def build_sound_collection(sound_files_path, csv_filename):
    # Make a Pandas DataFrame with the metadata of our sound collection and save it
    tracks = [f for f in listdir(sound_files_path) if isfile(join(sound_files_path, f)) and is_sound(join(sound_files_path, f))]
    df =  pd.DataFrame([make_pandas_record(tr, DATASET_DIR) for tr in tracks])
    df.sort_values('track_id', inplace=True) #alphabetically

    df.to_csv(csv_filename)
    print('Saved DataFrame with {0} entries! {1}'.format(len(df), csv_filename))    
    return df, tracks

df_tracks, _ = build_sound_collection(DATASET_LOCAL_DIR, DATAFRAME_CSV_TRACKS)
    
df_tracks.head(5)

Saved DataFrame with 90 entries! ./files/tracks.csv


,absolute_path,path,track_id
46,/Users/lluissuros/Documents/Datasets/phonos_da...,./phonos_dataset_local/all_alone.wav,all_alone.wav
18,/Users/lluissuros/Documents/Datasets/phonos_da...,./phonos_dataset_local/arpegio1_19_07.aiff,arpegio1_19_07.aiff
66,/Users/lluissuros/Documents/Datasets/phonos_da...,./phonos_dataset_local/arpegio1_filtered_lower...,arpegio1_filtered_lowered.wav
59,/Users/lluissuros/Documents/Datasets/phonos_da...,./phonos_dataset_local/arpegio1_impro.aiff,arpegio1_impro.aiff
60,/Users/lluissuros/Documents/Datasets/phonos_da...,./phonos_dataset_local/chords_ethereal_mallets...,chords_ethereal_mallets.wav


Save collection to json:

In [92]:
def create_tracks_dict(tracks_df, labels):
    print('creating tracks dictionary ...')
    track_ids = tracks_df.loc[:,'track_id'].values
    tracks = tracks_df.set_index('track_id')
    tracks_to_path = {}
    for track_id in track_ids:
        tracks_to_path[track_id] = tracks.loc[track_id, 'absolute_path']
        

    #df_clean = tracks_df.loc[:, labels] # only desired labels
    #tracks_to_path = df_clean.to_dict('index')
    
    print(len(tracks_to_path.keys()), '... by_frame_id entries created!')
    return tracks_to_path


def save_dict_to_json(dictionary, filename = './files/no_name.json'):
    print('saving dict ...')
    with open(filename, 'w') as fp:
        json.dump(dictionary, fp, sort_keys=True, indent=4) #pretty json
    print('... dict saved as', filename)
    
def load_json_dict(filename = './files/no_name.json'):
    with open(filename) as json_file:  
        data = json.load(json_file)
    return data
    
    
tracks_to_path = create_tracks_dict(df_tracks, ['absolute_path', 'track_id'])
save_dict_to_json(tracks_to_path,'./files/tracks_to_path.json')


## analyse sounds:
# TODO: I would like to analysie PITCH, see freesound exercise

In [9]:
# Define here our sound analysis function
# NOTE: remember that if you update this function and want to run a new analysis you'll need to re-run both 
# this cell and the cells below that carry out the audio analysis and that call the analysis function. 
# After analyzing the source collection or the target file, make sure to correct descriptors have been 
# extracted by checking the DataFrame contents. DataFrame contents can be printed on screen as a table 
# using 'display(data_frame_object)'


#TODO: at the moment there is not hop-size!!!
def analyze_sound(audio_path, absolute_path='missing', frame_size=None, audio_id=None):
    """Analyze the audio file given in 'sound_path'.
    Use the parameter 'frame_size' to set the size of the chunks in which the audio will 
    be split for analysis. If no frame_size is given, the whole audio will be analyzed as 
    a single frame.
    Use the 'audio_id' parameter to pass a custom identifier for the audio that will be 
    included in the analysis results. This can be useful to later identify to which file an analysis belongs.
    """
    analysis_output = []  # Here we'll store the analysis results for each chunk (frame) of the audio file
    
    # Load audio file
    #loader = estd.MonoLoader(filename=audio_path)
    loader = estd.EqloudLoader(filename=audio_path) #normalizes gain 

    audio = loader()
    
    # Some processing of frame_size parameter to avoid later problems
    if frame_size is None:
        frame_size = len(audio)  # If no frame_size is given use no frames (analyze all audio at once)
    if frame_size % 2 != 0:
        frame_size = frame_size + 1 # Make frame size even
    
    # Calculate the start and end samples for each equally-spaced audio frame
    frame_start_samples = range(0, len(audio), frame_size)
    frame_start_end_samples = zip(frame_start_samples[:-1], frame_start_samples[1:])

    # extract key and scale
    key_algo = estd.KeyExtractor()
    key, scale, key_strength = key_algo(audio)
    
    # Loudness extractor
    loudness_algo = estd.Loudness()
    
    # MFCC coefficients extractor
    w_algo = estd.Windowing(type = 'hann')
    spectrum_algo = estd.Spectrum()
    mfcc_algo = estd.MFCC()
    
    
    # Iterate over audio frames and analyze each one
    for count, (fstart, fend) in enumerate(frame_start_end_samples):
        
        # Get corresponding audio chunk and initialize dictionary to store analysis results with some basic metadata
        frame = audio[fstart:fend]
        frame_output = {
            'track_id': audio_id,
            'frame_id': '{0}_f{1}'.format(audio_id, count),
            'path': audio_path,
            'absolute_path': absolute_path,
            'start_sample': fstart,
            'end_sample': fend,
        }
        
        # Extract loudness
        loudness = loudness_algo(frame)
        frame_output['loudness'] = loudness / len(frame)  # Normnalize by length of frame

        # Extract MFCC coefficients
        spec = spectrum_algo(w_algo(frame))
        _, mfcc_coeffs = mfcc_algo(spec)
        frame_output.update({'mfcc_{0}'.format(j): mfcc_coeffs[j] for j in range(0, len(mfcc_coeffs))})
        
        # Other tonal features
        key, scale, key_strength = key_algo(frame)
        frame_output.update({'scale': scale,
                             'key_strength': key_strength})

        
        # Add frame analysis results to output
        analysis_output.append(frame_output)

    return analysis_output
    

## Analyze source collection

In [10]:
#DATAFRAME_FILENAME = 'dataframe.csv'  # DataFrame file of the sound source collection to analyze
#DATAFRAME_SOURCE_FILENAME = 'dataframe_source.csv'  # DataFrame file where to store the results of our analysis
#FRAME_SIZE = 8820

#TODO: at the moment there is not hop-size!!! 
def analyze_source_collection(dataframe_source, dataframe_results, frame_size=8820):
    #print('frame size ', frame_size )
    # Load the DataFrame of the sound source collection created in previous notebook and analyze all sound files in it
    df = pd.read_csv(open(dataframe_source), index_col=0)
    analyses = []
    for i in range(0, len(df)):
        sound = df.iloc[i]  # Get DataFrame sound at position 'i'
        print('Analyzing sound with id {0} [{1}/{2}]'.format(sound['track_id'], i + 1, len(df)))
        analysis_output = analyze_sound(sound['path'], sound['absolute_path'], frame_size=frame_size, audio_id=sound['track_id'])  # Split audio in chunks of 200ms (44100/5 samples)
        analyses += analysis_output

    # Store analysis results in a new Pandas DataFrame and save it
    df_source = pd.DataFrame(analyses)
    df_source.to_csv(dataframe_results)
    print('Saved source DataFrame with {0} entries! {1}'.format(len(df_source), dataframe_results))

    display(df_source)  # Show DataFrane contents
    df_source.describe()  # Show some statistics of numerical fields in the DataFrame
    return df


#analyze_source_collection(DATAFRAME_FILENAME, DATAFRAME_SOURCE_FILENAME)



## Analyze sources files and the target sound file

In [11]:
FRAME_SIZE_LOW_LEVEL = 2048
FRAME_SIZE_TONAL = 4096 #TODO how to deal with different frame sizes when bulking on the same dataframe

# DataFrame file where to store the results of our analysis
DATAFRAME_CSV_ANALYSIS = './files/tracks_analysis.csv'  

analyze_source_collection(
    DATAFRAME_CSV_TRACKS,
    DATAFRAME_CSV_ANALYSIS, 
    FRAME_SIZE_TONAL)


df = pd.read_csv(DATAFRAME_CSV_ANALYSIS) 


Analyzing sound with id all_alone.wav [1/90]
Analyzing sound with id arpegio1_19_07.aiff [2/90]
Analyzing sound with id arpegio1_filtered_lowered.wav [3/90]
Analyzing sound with id arpegio1_impro.aiff [4/90]
Analyzing sound with id chords_ethereal_mallets.wav [5/90]
Analyzing sound with id chords_funk_clav_3.wav [6/90]
Analyzing sound with id chords_organ.wav [7/90]
Analyzing sound with id chords_organ2.wav [8/90]
Analyzing sound with id chords_realistic_marimba.wav [9/90]
Analyzing sound with id chords_realistic_vibraphone.wav [10/90]
Analyzing sound with id chords_reverse_engineering.wav [11/90]
Analyzing sound with id chords_short_worm.wav [12/90]
Analyzing sound with id chords_solid_clav.wav [13/90]
Analyzing sound with id crotale01.aiff [14/90]
Analyzing sound with id crotale02.aiff [15/90]
Analyzing sound with id crotaleBrakeDrum01.aiff [16/90]
Analyzing sound with id crotaleBrakeDrum02.aiff [17/90]
Analyzing sound with id crotaleBrakeDrum03.aiff [18/90]
Analyzing sound with id c

,absolute_path,end_sample,frame_id,key_strength,loudness,mfcc_0,mfcc_1,mfcc_10,mfcc_11,mfcc_12,...,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,path,scale,start_sample,track_id
0,/Users/lluissuros/Documents/Datasets/phonos_da...,4096,all_alone.wav_f0,0.769967,0.000000e+00,-1138.420044,0.000011,-0.000027,-0.000031,-0.000038,...,-0.000061,0.000023,-0.000027,-0.000053,-0.000046,-0.000023,./phonos_dataset_local/all_alone.wav,major,0,all_alone.wav
1,/Users/lluissuros/Documents/Datasets/phonos_da...,8192,all_alone.wav_f1,0.769967,0.000000e+00,-1138.420044,0.000011,-0.000027,-0.000031,-0.000038,...,-0.000061,0.000023,-0.000027,-0.000053,-0.000046,-0.000023,./phonos_dataset_local/all_alone.wav,major,4096,all_alone.wav
2,/Users/lluissuros/Documents/Datasets/phonos_da...,12288,all_alone.wav_f2,0.769967,0.000000e+00,-1138.420044,0.000011,-0.000027,-0.000031,-0.000038,...,-0.000061,0.000023,-0.000027,-0.000053,-0.000046,-0.000023,./phonos_dataset_local/all_alone.wav,major,8192,all_alone.wav
3,/Users/lluissuros/Documents/Datasets/phonos_da...,16384,all_alone.wav_f3,0.769967,0.000000e+00,-1138.420044,0.000011,-0.000027,-0.000031,-0.000038,...,-0.000061,0.000023,-0.000027,-0.000053,-0.000046,-0.000023,./phonos_dataset_local/all_alone.wav,major,12288,all_alone.wav
4,/Users/lluissuros/Documents/Datasets/phonos_da...,20480,all_alone.wav_f4,0.769967,1.407930e-08,-1138.420044,0.000011,-0.000027,-0.000031,-0.000038,...,-0.000061,0.000023,-0.000027,-0.000053,-0.000046,-0.000023,./phonos_dataset_local/all_alone.wav,major,16384,all_alone.wav
5,/Users/lluissuros/Documents/Datasets/phonos_da...,24576,all_alone.wav_f5,0.769967,4.257478e-07,-1138.420044,0.000011,-0.000027,-0.000031,-0.000038,...,-0.000061,0.000023,-0.000027,-0.000053,-0.000046,-0.000023,./phonos_dataset_local/all_alone.wav,major,20480,all_alone.wav
6,/Users/lluissuros/Documents/Datasets/phonos_da...,28672,all_alone.wav_f6,0.769967,1.361909e-04,-863.024780,81.439682,-3.398426,-6.535385,-3.510933,...,-21.186806,-17.222984,-10.254883,-3.864639,-3.071655,-4.007729,./phonos_dataset_local/all_alone.wav,major,24576,all_alone.wav
7,/Users/lluissuros/Documents/Datasets/phonos_da...,32768,all_alone.wav_f7,0.769967,1.968472e-04,-905.911377,152.110229,-11.080568,-7.572319,-8.917149,...,11.227966,-32.111950,-20.426003,-15.404663,-4.562515,-6.192909,./phonos_dataset_local/all_alone.wav,major,28672,all_alone.wav
8,/Users/lluissuros/Documents/Datasets/phonos_da...,36864,all_alone.wav_f8,0.769967,1.040105e-04,-951.795532,138.362640,-3.379787,-4.338333,-10.570187,...,6.540604,-30.617031,-16.563423,-15.325462,-6.406471,3.312866,./phonos_dataset_local/all_alone.wav,major,32768,all_alone.wav
9,/Users/lluissuros/Documents/Datasets/phonos_da...,40960,all_alone.wav_f9,0.769967,7.020839e-05,-978.461060,122.226204,-4.884258,-7.302551,-10.952396,...,-2.140732,-32.791363,-11.491943,-15.401901,-8.053726,1.804672,./phonos_dataset_local/all_alone.wav,major,36864,all_alone.wav


# DOUBT: MAybe I need to scale mfccs before clustering ?

======================
## Standardize the Data
## TODO: is it really needed? --> understand the data!


In [18]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import sklearn

#1: Create a dataframe with the features (no targets and no categorical variables)
features = ['mfcc_0', 'mfcc_1', 'mfcc_2' , 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12']
# Separating out the features
x = df.loc[:, features].values


#2: standarize 
x = StandardScaler().fit_transform(x) # Standardizing the features
pd.DataFrame(data = x, columns = features).head() #visualize

,mfcc_0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12
0,-1.859299,-1.435602,0.451983,-0.79897,0.440276,0.802864,0.370693,0.730224,0.689491,0.753333,0.690114,0.590466,0.611691
1,-1.859299,-1.435602,0.451983,-0.79897,0.440276,0.802864,0.370693,0.730224,0.689491,0.753333,0.690114,0.590466,0.611691
2,-1.859299,-1.435602,0.451983,-0.79897,0.440276,0.802864,0.370693,0.730224,0.689491,0.753333,0.690114,0.590466,0.611691
3,-1.859299,-1.435602,0.451983,-0.79897,0.440276,0.802864,0.370693,0.730224,0.689491,0.753333,0.690114,0.590466,0.611691
4,-1.859299,-1.435602,0.451983,-0.79897,0.440276,0.802864,0.370693,0.730224,0.689491,0.753333,0.690114,0.590466,0.611691


# PCA: now disabled


## visualize 2D PCA for 3 different samples
### TODO: This scattor plot doesnt have the features scaled (use MinMaxScaler)

We can see below, that with MFCCs all the features are quite needed, as PCA to 95% needs 11 features. This would mean thta all features are important in explaining data variance (they are not correlated)

## Plot audio and features

In [26]:
#utilities for pandas

#retrieve freesound_id for a given audio file path
def get_fs_id_from_path(df, sound_path):
    return df.loc[df['path'] == sound_path].iloc[0]['freesound_id']

def feature_values_by_path(df, sound_path, feature):
    return df.loc[df['path'] == sound_path][feature].values


#TODO do it with external library if possible?
def scale_list(mylist):
    '''normalize against the maximum'''
    maximum = max([abs(min(mylist)), abs(max(mylist))])
    return [float(i)/maximum for i in mylist]




# TODO: instead of scaling for the plot, maybe is it worth to sclae them after computing pca and leaving them sclaed all along?

## PLOT clusters and centroids with k-means
#### TODO Is is better to PCA after or before clustering?

In [31]:
## creating the non-feature dataset for index reference:
other_columns = ['start_sample', 'end_sample', 'path', 'absolute_path', 'track_id', 'frame_id'] 
# Separating out the target,  mantain the files info for reference later:
y = df.loc[:, other_columns].values

info_df = pd.DataFrame(data = y, columns = other_columns) #visualize
print(info_df.shape)
info_df.head()

(88481, 6)


,start_sample,end_sample,path,absolute_path,track_id,frame_id
0,0,4096,./phonos_dataset_local/all_alone.wav,/Users/lluissuros/Documents/Datasets/phonos_da...,all_alone.wav,all_alone.wav_f0
1,4096,8192,./phonos_dataset_local/all_alone.wav,/Users/lluissuros/Documents/Datasets/phonos_da...,all_alone.wav,all_alone.wav_f1
2,8192,12288,./phonos_dataset_local/all_alone.wav,/Users/lluissuros/Documents/Datasets/phonos_da...,all_alone.wav,all_alone.wav_f2
3,12288,16384,./phonos_dataset_local/all_alone.wav,/Users/lluissuros/Documents/Datasets/phonos_da...,all_alone.wav,all_alone.wav_f3
4,16384,20480,./phonos_dataset_local/all_alone.wav,/Users/lluissuros/Documents/Datasets/phonos_da...,all_alone.wav,all_alone.wav_f4


In [34]:
#https://musicinformationretrieval.com/kmeans.html
from sklearn.cluster import KMeans
from itertools import cycle
from matplotlib import colors


'''
TODO reuse commented code above to PCA

PCA_components = finalDf[['principal component 1', 'principal component 2']].values
print(PCA_components.shape)
features_2D = PCA_components # like this wecan change PCA for any other thing
'''

N_WORDS = 256


features = [ 'mfcc_0', 'mfcc_1', 'mfcc_2' , 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12']
features_df = df.loc[:, features].values # Separating out the features

print("creating codebook of ", N_WORDS, " words, might some minutes, specially with high number of words")
print("centroids (words) will have", len(features) , " dimensions")


#time to cluster! with k-means
k_means = KMeans(n_clusters=N_WORDS)
k_means.fit_predict(features_df)
labels = k_means.labels_ 
centroids = k_means.cluster_centers_ #take the cluster center

print("codebook finished! ")


print(labels)
print(centroids)




creating codebook of  256  words, might take a while
centroids (words) will have 13  dimensions
[153 153 153 ...,  61  61  61]
[[ -9.45000138e+02   1.45689647e+02   1.14172927e+01 ...,  -1.91705067e+01
   -1.63741794e+01  -1.19883360e+01]
 [ -1.10078790e+03  -8.33224140e+00  -2.78036945e+01 ...,   1.55579245e+00
    8.99430995e-01  -1.72932262e+00]
 [ -8.45449390e+02   1.40609598e+02  -4.18185954e+01 ...,  -1.48720422e+01
   -1.26104950e+01  -1.64778088e+01]
 ..., 
 [ -8.34593336e+02   1.48781018e+02  -2.49624176e+01 ...,  -7.79415925e+00
   -6.57128639e+00  -5.17824666e+00]
 [ -8.67801241e+02   2.08445657e+02  -4.73097583e+00 ...,  -1.45127736e+01
   -1.13681550e+01  -5.53031556e+00]
 [ -7.69504966e+02   2.06955622e+02  -6.98500765e+01 ...,  -1.08378569e+01
   -5.27892040e+00  -7.43957836e+00]]


### Save the codebook
Like this, we dont need to recompute all the time if we don't want to 
#### NOTE: Remember to convert it to numpy arrays when loading

In [104]:
# Save Codebook

CODEBOOK_FILENAME = './files/codebook.json'
codebook_for_save = {'centroids': centroids.tolist(), 'labels': labels.tolist()}

save_dict_to_json(codebook_for_save, CODEBOOK_FILENAME)

#test
'''saved_dict = load_json_dict(CODEBOOK_FILENAME)
test_centroids = np.array(saved_dict['centroids'])
test_labels = np.array(saved_dict['labels'])'''

print('\n codebook is saved as ', CODEBOOK_FILENAME)



saving dict ...
... dict saved as ./files/codebook.json

codebook is saved as  ./files/codebook.json


Load Codebook (very imporatnt if we don't want to recompute): 

In [106]:
codebook_saved_dict = load_json_dict(CODEBOOK_FILENAME)

centroids = np.array(saved_dict['centroids'])
labels = np.array(saved_dict['labels'])


print('\n codebook is loaded from disc', CODEBOOK_FILENAME)


 codebook is loaded from disc ./files/codebook.json


### TODO : Now we can get the codeword label and add it to the dataframe, analysis csv and the json!

# TODO: It is in here, after clustering, where we could PCA to plot


## Encode tracks with the obtained CODEBOOK, and compute histograms
Will create two dictionaries by_id, one with the encoded frames, and other with the histogram of this encode frames.
An histogram is a graphical representation of the value distribution.


### We are gettting the 3 nearest centroids at the moment:
The 1-nn is the codeword for that frame!!
We are also appending the new info to the analysis dataframe ans saving it.


## TODO: If interested in histograms, should we use 1-nn?


In [234]:
from sklearn.neighbors import NearestNeighbors


def create_histograms(encoded_tracks_by_id, codebook):
    '''returns a dictionary with id and the corresponding histogram of encoded frames'''
    bins = create_bins_for_histogram(codebook)
    histograms_by_id = {} 
    for track_id in encoded_tracks_by_id.keys():
        histogram, _ = np.histogram(encoded_tracks_by_id[track_id], bins)
        histograms_by_id[track_id] = histogram
    return histograms_by_id


def create_bins_for_histogram(codebook):
    '''
    https://stackoverflow.com/questions/30112420/histogram-for-discrete-values-with-matplotlib
    the default bins will not be centered around the integer: 
    ... so the trick is to set up the bins centered on the integers, i.e.
    -0.5, 0.5, 1,5, 2.5, ... up to max(data) + 1.5. Then you substract -0.5 to eliminate the extra bin at the end.
    '''
    return np.arange(0, len(codebook) + 1.5) - 0.5


def encode_all_audios(codebook, dataframe, features):
    '''
    Codebook: array of n-dimensional points corresponding to codewords (cluster centroids)
       
    Will encode all audios, finding which is the nearest cluster centroid(code vector) for each frame.
    1-nearest-neighbour is used to get the closest centroid.
    Returns a dictionary by id and the corresponding array of encoded frames.
    '''
    nbrs = NearestNeighbors(n_neighbors=N_NEIGHBORS, algorithm='ball_tree').fit(centroids) 
    unique_ids = dataframe['track_id'].unique() #get unique ids
    encoded_tracks_by_id = {}
    for count,track_id in enumerate(unique_ids):    
        encoded_track = encode_track_frames(track_id, dataframe, features, nbrs)
        encoded_tracks_by_id[track_id] = encoded_track
        #print('encoded track ', track_id , ' , number:' , count, '/', len(unique_ids))
    return encoded_tracks_by_id


def encode_track_frames(track_id, dataframe, features, nbrs):
    file_frames = dataframe.loc[dataframe['track_id'] == track_id] # frames belonging the track_id
    frames_values = file_frames.loc[:, features].values
    _, indices = nbrs.kneighbors(frames_values) #obtain the nearest centroid for each frame on the input
    indices = indices.squeeze() # squeeze will get rid of unnecesary dimesions
    return indices 



N_NEIGHBORS = 3
print('encoding, getting ', N_NEIGHBORS,  '-nn... \n')



#features used in encoding 
#TODO: dry, only one place!!!!
features = [ 'mfcc_0', 'mfcc_1', 'mfcc_2' , 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12']

#encode all tracks:
encoded_tracks_by_id = encode_all_audios(centroids, analysis_df, features)    
print('encoded_tracks_by_id was computed')

#create histograms:
histograms_by_id = create_histograms(encoded_tracks_by_id, centroids)
print('histograms_by_id was computed')


encoding, getting  3 -nn... 

encoded_tracks_by_id was computed
histograms_by_id was computed


### Add nearest neighbours columns, to save it later in the json

In [284]:
def add_nearest_neighbours_columns(dataframe, encoded_tracks_by_id):
    #Add new columns to dataframe:
    for column in range(N_NEIGHBORS):
        new_column = 'word_' + str(column+1) +  '_nearest'
        dataframe[new_column] = 0 
        print('Add new column:' , new_column)

    #fill values for each track:
    for track_id in dataframe['track_id'].unique():
        idx = dataframe.index[dataframe['track_id'] == track_id]
        for n_neighbour in range(N_NEIGHBORS):
            nearest_n_column = 'word_'+ str(n_neighbour+1) +'_nearest'
            dataframe.at[idx, nearest_n_column] = encoded_tracks_by_id[track_id][:,n_neighbour]
        
        
add_nearest_neighbours_columns(analysis_df, encoded_tracks_by_id)

analysis_df.head(10)


Add new column: word_1_nearest
Add new column: word_2_nearest
Add new column: word_3_nearest


,Unnamed: 0,absolute_path,end_sample,frame_id,key_strength,loudness,mfcc_0,mfcc_1,mfcc_10,mfcc_11,...,mfcc_7,mfcc_8,mfcc_9,path,scale,start_sample,track_id,word_1_nearest,word_2_nearest,word_3_nearest
88461,88461,/Users/lluissuros/Documents/Datasets/phonos_da...,131072,whale9.wav_f31,0.21663,0.000064,-933.039856,29.559753,-2.954151,-0.819229,...,-11.106056,-4.273882,-6.109180,./phonos_dataset_local/whale9.wav,minor,126976,whale9.wav,61,30,204
88462,88462,/Users/lluissuros/Documents/Datasets/phonos_da...,135168,whale9.wav_f32,0.21663,0.000042,-939.216858,35.649025,-4.932209,-6.807301,...,-10.216316,-0.363819,-4.915638,./phonos_dataset_local/whale9.wav,minor,131072,whale9.wav,61,30,204
88463,88463,/Users/lluissuros/Documents/Datasets/phonos_da...,139264,whale9.wav_f33,0.21663,0.000030,-962.369141,37.719727,0.022896,-5.464924,...,-12.993843,-1.087635,-4.591125,./phonos_dataset_local/whale9.wav,minor,135168,whale9.wav,61,113,30
88464,88464,/Users/lluissuros/Documents/Datasets/phonos_da...,143360,whale9.wav_f34,0.21663,0.000030,-959.479309,39.478104,-0.601910,-6.720280,...,-7.289207,-0.024548,-6.713272,./phonos_dataset_local/whale9.wav,minor,139264,whale9.wav,61,30,113
88465,88465,/Users/lluissuros/Documents/Datasets/phonos_da...,147456,whale9.wav_f35,0.21663,0.000042,-933.784424,32.525608,-2.840775,-3.684895,...,-4.717400,-5.809738,-9.968224,./phonos_dataset_local/whale9.wav,minor,143360,whale9.wav,61,204,30
88466,88466,/Users/lluissuros/Documents/Datasets/phonos_da...,151552,whale9.wav_f36,0.21663,0.000053,-921.401367,38.643883,0.572670,2.786362,...,3.536446,8.899689,-6.736671,./phonos_dataset_local/whale9.wav,minor,147456,whale9.wav,61,204,30
88467,88467,/Users/lluissuros/Documents/Datasets/phonos_da...,155648,whale9.wav_f37,0.21663,0.000041,-916.344360,39.435402,0.934422,-4.605137,...,-6.525497,-2.891972,1.025723,./phonos_dataset_local/whale9.wav,minor,151552,whale9.wav,61,204,243
88468,88468,/Users/lluissuros/Documents/Datasets/phonos_da...,159744,whale9.wav_f38,0.21663,0.000037,-951.001648,43.019104,0.412342,-0.735973,...,-10.524258,-3.921719,-4.870564,./phonos_dataset_local/whale9.wav,minor,155648,whale9.wav,61,30,113
88469,88469,/Users/lluissuros/Documents/Datasets/phonos_da...,163840,whale9.wav_f39,0.21663,0.000030,-955.794556,36.549889,-2.961689,-6.663754,...,-9.537815,4.301918,1.450932,./phonos_dataset_local/whale9.wav,minor,159744,whale9.wav,61,113,30
88470,88470,/Users/lluissuros/Documents/Datasets/phonos_da...,167936,whale9.wav_f40,0.21663,0.000040,-941.086914,46.091293,-6.877720,-8.398182,...,-14.699425,-7.145054,-9.367687,./phonos_dataset_local/whale9.wav,minor,163840,whale9.wav,61,30,113


## create by_frame_id dictionary
Instead of a csv, it will be much more confortable to deal with Objects in superCollider, so I will create the jsons:

## TODO this comes after codebook
## TODO next and previous frame
## TODO nearest_1_centroid, nearest_2_centroid, nearest_3_centroid
## TODO: more fields could come later like embeddings, most_similars, codeword, knn(3), pitch ...

In [12]:
def create_by_frame_id_dict(tracks_df, analysis_df, labels):
    print('creating by_frame_id ...')
    
    track_ids = tracks_df['track_id'].unique() #just in case
    df_clean = analysis_df.loc[:, labels] # only desired labels
    df_clean.set_index('frame_id', inplace=True)
    by_frame_id = df_clean.to_dict('index')
    
    print(len(by_frame_id.keys()), '... by_frame_id entries created!')
    return by_frame_id


def save_dict_to_json(dictionary, filename = './files/no_name.json'):
    print('saving dict ...')
    with open(filename, 'w') as fp:
        json.dump(dictionary, fp, sort_keys=True, indent=4) #pretty json
    print('... dict saved as', filename)
    
    
# list(df.columns.values) # all labels

columns_to_keep = [
 'absolute_path',   
 'end_sample',
 'frame_id',
 'loudness',
 'mfcc_0',
 'mfcc_1',
 'mfcc_10',
 'mfcc_11',
 'mfcc_12',
 'mfcc_2',
 'mfcc_3',
 'mfcc_4',
 'mfcc_5',
 'mfcc_6',
 'mfcc_7',
 'mfcc_8',
 'mfcc_9',
 'word_1_nearest',
 'word_2_nearest',
 'word_3_nearest'
# 'path',
 'scale',
 'start_sample',
 'track_id']


BY_FRAME_ID_JSON = './files/by_frame_id.json'
tracks_df = pd.read_csv(DATAFRAME_CSV_TRACKS) 
analysis_df = pd.read_csv(DATAFRAME_CSV_ANALYSIS)

by_frame_id = create_by_frame_id_dict(tracks_df, analysis_df, columns_to_keep)

save_dict_to_json(by_frame_id, BY_FRAME_ID_JSON)

test_id = 'spot1.wav_f1146'
print('\n test frame_id: ', test_id)
by_frame_id[test_id] #test


creating by_frame_id ...
88481 ... by_frame_id entries created!
saving dict ...
... dict saved as ./files/by_frame_id.json

 test frame_id: 


{'absolute_path': '/Users/lluissuros/Documents/Datasets/phonos_dataset/spot1.wav',
 'end_sample': 4698112,
 'loudness': 0.0008997643599286675,
 'mfcc_0': -904.8228759765624,
 'mfcc_1': 191.4112243652344,
 'mfcc_10': -12.457365036010742,
 'mfcc_11': -25.915771484375,
 'mfcc_12': -12.29935073852539,
 'mfcc_2': 24.12707901000977,
 'mfcc_3': 1.7669563293457031,
 'mfcc_4': -28.39010238647461,
 'mfcc_5': -50.58886337280274,
 'mfcc_6': -25.315338134765625,
 'mfcc_7': -24.70161437988281,
 'mfcc_8': -15.87053680419922,
 'mfcc_9': 4.792762756347656,
 'scale': 'minor',
 'start_sample': 4694016,
 'track_id': 'spot1.wav'}

=====================


### Plot some histograms:
https://matplotlib.org/gallery/subplots_axes_and_figures/subplots_demo.html


In [ ]:
#test_id = 232335

plot_ids = finalDf['freesound_id'].unique()[:4] #get 4 unique ids to plot

#for plotting, we use matplotlib instead on numpy.histogram:
fig, axs = plt.subplots(2, 2, figsize=(20, 10))
fig.suptitle('Plotting 4 histograms', fontsize=32)

axs[0, 0].hist(encoded_tracks_by_id[plot_ids[0]], bins, color = "skyblue")  
axs[0, 0].set_title('freesound_id {}'.format(plot_ids[0]))
axs[0, 0].set_xticks(bins + 0.5)
axs[0, 1].hist(encoded_tracks_by_id[plot_ids[1]], bins, color = "orange")  
axs[0, 1].set_title('freesound_id {}'.format(plot_ids[1]))
axs[0, 1].set_xticks(bins + 0.5)
axs[1, 0].hist(encoded_tracks_by_id[plot_ids[2]], bins, color = "green")  
axs[1, 0].set_title('freesound_id {}'.format(plot_ids[2]))
axs[1, 0].set_xticks(bins + 0.5)
axs[1, 1].hist(encoded_tracks_by_id[plot_ids[3]], bins, color = "red")
axs[1, 1].set_title('freesound_id {}'.format(plot_ids[3]))
axs[1, 1].set_xticks(bins + 0.5)

for ax in axs.flat:
    ax.set(xlabel='codewords are numbers', ylabel='frequency')


# Compare histograms

https://stats.stackexchange.com/questions/7400/how-to-assess-the-similarity-of-two-histograms

### Approach 1: intersection 
simple!


In [ ]:
#https://mpatacchiola.github.io/blog/2016/11/12/the-simplest-classifier-histogram-intersection.html

def return_intersection(hist_1, hist_2):
    minima = np.minimum(hist_1, hist_2)
    intersection = np.true_divide(np.sum(minima), np.sum(hist_2))
    return intersection

test_ids = finalDf['freesound_id'].unique()[:4] #get 4 unique ids to test

print (return_intersection(histograms_by_id[test_ids[0]], histograms_by_id[test_ids[0]]))
print (return_intersection(histograms_by_id[test_ids[0]], histograms_by_id[test_ids[1]]))
print (return_intersection(histograms_by_id[test_ids[2]], histograms_by_id[test_ids[3]])) 


### Approach 2: Cosine similarity
Cosine similarity is a metric used to measure how similar the documents are irrespective of their size. Mathematically, it measures the cosine of the angle between two vectors projected in a multi-dimensional space.

https://www.machinelearningplus.com/nlp/cosine-similarity/

## TODO: Soft Cosine Similarity?
If you want a similarity metric that gives higher scores for documents belonging to the same topic and lower scores when comparing docs from different topics.

In [ ]:
# Define the documents
doc_trump = "Mr. Trump became president after winning the political election. Though he lost the support of some republican friends, Trump is friends with President Putin"

doc_election = "President Trump says Putin had no political interference is the election outcome. He says it was a witchhunt by political parties. He claimed President Putin is a friend who had nothing to do with the election"

doc_putin = "Post elections, Vladimir Putin became President of Russia. President Putin had served as the Prime Minister earlier in his political career"

documents = [doc_trump, doc_election, doc_putin]


#test: treat encoded vectors as text documents!



# Scikit Learn
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

'''
NOTE:
Even better, I could have used the TfidfVectorizer() instead of CountVectorizer(), 
because it would have downweighted words that occur frequently across docuemnts.'''


## What I could do, is to provd 

# Create the Document Term Matrix
#count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(documents)

# OPTIONAL: Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=['doc_trump', 'doc_election', 'doc_putin'])
df

# Compute Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(df, df))
#> [[ 1.          0.48927489  0.37139068]
#>  [ 0.48927489  1.          0.38829014]
#>  [ 0.37139068  0.38829014  1.        ]]

#print(sparse_matrix)
df.shape

### TD IDF: obtain similarity matrix
## TODO:  I could have used the TfidfVectorizer() instead of CountVectorizer()

https://www.machinelearningplus.com/nlp/cosine-similarity/

Also more info in: https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

In [ ]:
#https://www.machinelearningplus.com/nlp/cosine-similarity/
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

'''
TODO:
Even better, I could have used the TfidfVectorizer() instead of CountVectorizer(), 
because it would have downweighted words that occur frequently across docuemnts.
'''


def convert_encoded_tracks_as_text(encoded_tracks_by_id):
    encoded_tracks_as_text_by_id = {}
    for track_id in encoded_tracks_by_id.keys():
        encoded_track_as_text = ' '.join('c' + str(e) for e in encoded_tracks_by_id[track_id])
        encoded_tracks_as_text_by_id[track_id] = encoded_track_as_text
    return encoded_tracks_as_text_by_id
    
encoded_tracks_as_text_by_id = convert_encoded_tracks_as_text(encoded_tracks_by_id)

#tf-idf algorithm requires to provide an array for texts:
track_ids = np.array([])
tracks_as_text = np.array([])
for track_id in encoded_tracks_as_text_by_id.keys():
    track_ids = np.append(track_ids, track_id)
    tracks_as_text = np.append(tracks_as_text, encoded_tracks_as_text_by_id[track_id])  

documents = tracks_as_text # this is our text corpus


# Create the Document Term Matrix
#count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(documents)

# OPTIONAL: Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=track_ids)

#print(sparse_matrix)

similarity_matrix = cosine_similarity(df) # Compute Cosine Similarity between rows
print(similarity_matrix.shape)
print(similarity_matrix)
df.head(10)


In [ ]:
'''
TODO
# https://pythonprogramminglanguage.com/kmeans-text-clustering/
# simple example for clustering right after tf-idf (without posterior cosine similarity matrix)

#we can even predict something next, see last lines in: 
https://github.com/MihailSalnikov/tf-idf_and_k-means/blob/master/main.ipynb

'''


## TF-IDF alternatives for text clustering?
https://datascience.stackexchange.com/questions/33227/what-approach-other-than-tf-idf-could-i-use-for-text-clustering-using-k-means
 In this response:
 - Like Tf-Idf, GloVe represents a group of words as a vector. Unlike Tf-Idf, which is a Bag-of-Words approach, GloVe and similar techniques preserve the order of words in a tweet. Knowing what word comes before or after a word of interest is valuable information for assigning meaning
 - For your clustering, I recommend checking out Density-Based clustering. K-means is a decent all-purpose algorithm, but it's a partitional method and depends on assumptions that might not be true, such as clusters being roughly equal in size.

Further links:


### word2vec:
In data clustering algorithms instead of bag of words (BOW) model we can use Word2Vec. The advantage of using Word2Vec is that it can capture the distance between individual words.

Each word will be converted in a vector of the specified bit length.
==> very easy to cluster "similar" words

seems easy to do as well: https://www.youtube.com/watch?v=thLzt3D-A10
 Gensim Python library
obviously, for our poutpose we need to train the model ourself, 
https://github.com/shreyans29/thesemicolon/blob/master/word2vec.py

can also get `mostsimilar`

### doubt: but then, can I cluster the documents? how?





----
Maybe this is also interesting!!!

(both clusterings are with k-means)

(document level) --> Text Clustering with doc2vec :
http://ai.intelligentonlinetools.com/ml/text-clustering-doc2vec-word-embedding-machine-learning/

(sentence level) --> Text Clustering with Word Embedding word2vec
http://ai.intelligentonlinetools.com/ml/text-clustering-word-embedding-machine-learning/

(otherwise, we would only be able to cluster at a word level, since words are n-dimesional vectors)

`Doc2Vec` (also called Paragraph Vectors) is an extension of Word2Vec, which learns the meaning of documents instead of words.
paper --> https://arxiv.org/pdf/1607.05368.pdf
interesting idea after clustering --> we counted the number of occurrences of each word. We then selected the 5 most occurring words per cluster as keywords for that cluster. This approach worked surprisingly well. 

---



check also
### 2.1.3 Co-Occurrence Matrix with a fixed context window 
https://www.analyticsvidhya.com/blog/2017/06/word-embeddings-count-word2veec/


In [ ]:
def sim_to_dist(m):
    '''
    converts similarity to distance matrix
    uses vectorized ufunc for better performance: 
    https://stackoverflow.com/questions/42594695/how-to-apply-a-function-map-values-of-each-element-in-a-2d-numpy-array-matrix?rq=1
    
    possible performance improvement here (we only need to compute the upper half on the diagonal):
    https://stackoverflow.com/questions/25650100/use
    º-a-similarity-function-for-clustering-scikit-learn 
    At the moment it is fast, so I didn't do it so far.
    '''
    return np.round((1 - m), 6)

distance_matrix = sim_to_dist(similarity_matrix) # faster using numpy built-in ufuncs
distance_matrix


## Hierarchical clustering and dendogram:
not sure if needed, but I leave the idea here just in case.

In [ ]:
# 6- Use a hierarchical clustering algorithm to cluster subgenres. 
#Visualize dendrograms (generate a huge high-resolution dendrogram image listing all analyzed subgenres). 
#This will be a very huge image to fit in the report, so it may be better to arrange it in a way that the subgenres are stacked vertically (or include it as an attachment pdf/png). 
#Consider showing just a few zoomed-in images of interesting genres.

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html
# https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.cluster.hierarchy.dendrogram.html

%matplotlib inline
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib 


dist = similarity_matrix 

#Perform hierarchical/agglomerative clustering.
Z = linkage(dist, optimal_ordering=True)

plt.figure(figsize=(300, 100))

min_threshold = 200
plt.title('Hierarchical clustering dendrogram of sub-genres on 4 different sources dataset using a threshold of minimum '
          +str(min_threshold), fontsize = 20)
plt.xlabel('sample index')
plt.ylabel('subgenre label')

#dendrogram(Z, labels=subgenres_to_keep)
dendrogram(
    Z,
#    labels=subgenres_to_keep,
    leaf_rotation=90.,
    leaf_font_size=8.,
    show_contracted=True,  # to get a distribution impression in truncated branches
)



#plt.savefig("dendogram.png", facecolor=('white'), edgecolor='none')
plt.show()

## A different aproach on hierarchical clustering and dendogram.
# TODO: correct labels on dendogram: 
in stack overflow the first option is provided but I don't understand why we do this : linkage = hcluster.linkage(1 - distVec)

In [ ]:
'''
A way to pass this similarity matrix through to the dendrogram so it plots correctly:
Turns out the SimMatrix needs to be first converted into a condensed matrix 
(the diagonal, upper right or bottom left, of this matrix):

https://stackoverflow.com/questions/29022451/dendrogram-through-scipy-given-a-similarity-matrix?rq=1
'''
import scipy.cluster.hierarchy as hcluster
import scipy.spatial.distance as ssd


fig = plt.figure(figsize = (20,8))
ax = fig.add_subplot(1, 1, 1)

distVec = ssd.squareform(distance_matrix)
linkage = hcluster.linkage(distVec)
dendro  = hcluster.dendrogram(linkage)

fig.suptitle('Closer in the dendogram, more similar files. TODO: put correct x-labels, should be the track_id', fontsize=32)
ax.tick_params(axis='y', which='major', labelsize=15)
ax.tick_params(axis='x', which='major', labelsize=15)
plt.show()


#fig.savefig('t.png')


print('very far away:')
print(similarity_matrix[16, 40])
print(similarity_matrix[16, 44])
print(similarity_matrix[44, 40])


print('not so similars:')
print(similarity_matrix[22, 49])
print(similarity_matrix[20, 1])
print(similarity_matrix[20, 55])
print(similarity_matrix[8, 37])
print(similarity_matrix[6, 40])
print(similarity_matrix[40, 6])
print(similarity_matrix[22, 25])

print('similars')
print(similarity_matrix[8, 58])
print(similarity_matrix[32, 37])
print(similarity_matrix[5, 50])
print(similarity_matrix[9, 29])
print(similarity_matrix[31, 53])
print(similarity_matrix[38, 59])




### heatmap with dendogram

https://python-graph-gallery.com/404-dendrogram-with-heat-map/

When you use a dendrogram to display the result of a cluster analysis, it is a good practice to add the corresponding heatmap. It allows you to visualise the structure of your entities (dendrogram), and to understand if this structure is logical (heatmap).  This is easy work thanks to the seaborn library that provides an awesome ‘cluster map’ function. This page aims to describe how it works, and note that once more the seaborn documentation is awesome.

In [ ]:
sns.clustermap(similarity_matrix, metric="euclidean", standard_scale=1, method="ward")

# Clustering tracks from the cosine distance similarity matrix:
https://stackoverflow.com/questions/30089675/clustering-cosine-similarity-matrix
https://towardsdatascience.com/spectral-clustering-for-beginners-d08b7d25b4d8

## TODO: Take a better look if I am doing this properly (providing correct matrix and hyperparameters)

## TODO: Can also use hierarchical clustering if the dataset is not too big (it is expensive)

`SpectralClustering`: using `precomputed`, a user-provided affinity matrix (similarity matrix) can be used.

For the cases you want the algorithm to figure out the number of clusters by itself, you can use Density Based Clustering Algorithms like DBSCAN:
 

In [ ]:
import numpy as np
from sklearn.cluster import SpectralClustering
#mat = np.matrix([[1.,.1,.6,.4],[.1,1.,.1,.2],[.6,.1,1.,.7],[.4,.2,.7,1.]])


clusters = SpectralClustering(20, affinity="precomputed").fit_predict(similarity_matrix) # we provide our precomputed affinity(similarity) matrix
print(len(clusters))
clusters


#### For the cases you want the algorithm to figure out the number of clusters by itself, you can use Density Based Clustering Algorithms like DBSCAN:

In [ ]:
'''
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html
 NOTE/TODO: (...)One way to avoid the query complexity is to pre-compute sparse neighborhoods in chunks using NearestNeighbors.radius_neighbors_graph with mode='distance', then using metric='precomputed' here.
'''
# Perform DBSCAN clustering from vector array or distance matrix
'''
TODO: check this as well: 
https://stackoverflow.com/questions/53194672/sklearn-dbscan-cosine-vs-precomputed
 Am I dealing with cosine distance correctly? In here says:

(...)The linear kernel and cosine distance are close mathematically 
but the linear kernel will give 1 for full similarity, whereas a cosine distance for full similarity is 0, so (...)
'''

#works with DISTANCE matrix (from documentation)
from sklearn.cluster import DBSCAN
DBSCAN(min_samples=1).fit_predict(distance_matrix)


# Affinity propagation:
# TODO: I think is not needed anymore
In scikit-learn, other clustering algorithms such as affinity propagation can cluster without defining the number of clusters beforehand.

TODO: All we need to do is swap out KMeans for AffinityPropagation:

### But crashes because to many PCA_components (until 3000 it "resists")

https://musicinformationretrieval.com/kmeans.html

In [ ]:
# AffinityPropagation and Plot result (https://scikit-learn.org/stable/auto_examples/cluster/plot_affinity_propagation.html)
from sklearn.cluster import AffinityPropagation
from itertools import cycle


af = AffinityPropagation()
X = PCA_components[:1000] #CRASHES if we use all
labels = af.fit_predict(X) 
cluster_centers_indices = af.cluster_centers_indices_
n_clusters_ = len(cluster_centers_indices)

#print(labels)
print('Estimated number of clusters: %d' % n_clusters_)

plt.close('all')
plt.figure(1)
plt.clf()

colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
for k, col in zip(range(n_clusters_), colors):
    class_members = labels == k
    cluster_center = X[cluster_centers_indices[k]]
    plt.plot(X[class_members, 0], X[class_members, 1], col + '.')
    plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
             markeredgecolor='k', markersize=14)
    for x in X[class_members]: # this part is computationally Very expensive, can kill kernel and is not really needed
        plt.plot([cluster_center[0], x[0]], [cluster_center[1], x[1]], col) #draw lines to centroid

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

## TODO (from Xavier):
 * codebook encoding
 * histogram ()
 * similarity function/matrix over the "codebook features" (varias opciones seguramente)
 * plot: plot features and their mean to show how different they are

next week with xavier:
 * degree clustering xavier technique

## SOTA:
 * Codebook:

https://www.sciencedirect.com/topics/engineering/vector-quantization 
Each input vector can be viewed as a point in an n-dimensional space. The vector quantizer is defined by a partition of this space into a set of nonoverlapping n-dimensional regions. The vector is encoded by comparing it with a codebook consisting of a set of stored reference vectors known as codevectors. 
 The optimality criterion is that a quantization region should consist of all vectors that are closer to its codevector than any of the other codevectors, and the codevector should be the average of all vectors that are in the quantization region.
 //Ali Grami, in Introduction to Digital Communications, 2016, chapter 5.2.3 Vector Quantization


Vector quantization (VQ) provides an efficient technique for data compression. Compression is achieved by transmitting the index of the codeword instead of the vector itself.
 VQ can be defined as a mapping that assigns each vector x=(x0,x1,…,xn-1)T in the n-dimensional space Rn to a codeword from a finite subset of Rn. The subset Y={yi:i=1,2,…,M} representing the set of possible reconstruction vectors is called a codebook of size M. Its members are called the codewords. In the encoding process, a distance measure is evaluated to locate the closest codeword for each input vector x. Then, the address corresponding to the codeword is assigned to x and transmitted. 
 A vector quantizer achieving a minimum encoding error is referred to as a Voronoi quantizer. Figure 7.9 shows an input data space partitioned into four different regions, called Voronoi cells, and the corresponding Voronoi vectors. These regions describe the collection of only those input vectors that are very close to the respective Voronoi vector.
 //Anke Meyer-Baese, Volker Schmid, in Pattern Recognition and Signal Analysis in Medical Imaging (Second Edition), 2014
 
 
  Thus the entire space Sb is divided into a finite number of cells and a code point is associated with each one. The code point is used to represent all of the points in that cell during the clustering process. The point with the smallest function value of a cell is the most suitable code point. Further, code points need not be sample points; they can be generated independently. They may also be centroids of the cells. Identification of a cluster is done using vector quantization of the reduced sample points. 
  // Jasbir Singh Arora, in Introduction to Optimum Design (Fourth Edition), 2017
 
 
 
 
 ----
 More on this but different:
 * Learnig Vector Quantisation: 
 Recent developments in neural network architectures have led to a new VQ concept, the so-called learning vector quantization (LVQ). It represents an unsupervised learning algorithm associated with a competitive neural network consisting of one input and one output layer. The algorithm permits only the update of the winning prototype, that is, the closest prototype (Voronoi vector) of the LVQ network.
  LVQ procedures are intuitively clear and easy to implement. The classification of data is based on a comparison with a number of so-called prototype vectors.
The relative simplicity of the LVQ and its ability to work in unsupervised mode have made it a useful tool for image segmentation problems [190]
 
 
 
 //k-means for example can be used for vector quantization, taking the centroid of each cluster as the codebook.
In computer vision, the bag-of-words model (BoW model) can be applied to image classification, by treating image features as words. In document classification, a bag of words is a sparse vector of occurrence counts of words; that is, a sparse histogram over the vocabulary. In computer vision, a bag of visual words is a vector of occurrence counts of a vocabulary of local image features.
https://en.wikipedia.org/wiki/Bag-of-words_model_in_computer_vision#cite_note-feifeicvpr2005-1
 
 
------- 
------- 
( 
https://machinelearningmastery.com/implement-learning-vector-quantization-scratch-python/
LVQ is a supervised version of vector quantization that can be used when we have labelled input data.
A limitation of k-Nearest Neighbors is that you must keep a large database of training examples in order to make predictions.

The Learning Vector Quantization algorithm addresses this by learning a much smaller subset of patterns that best represent the training data.
Predictions are made by finding the best match among a library of patterns. The difference is that the library of patterns is learned from training data, rather than using the training patterns themselves

(In LVQ) The library of patterns are called codebook vectors and each pattern is called a codebook. The codebook vectors are initialized to randomly selected values from the training dataset. Then, over a number of epochs, they are adapted to best summarize the training data using a learning algorithm.
 )



---
(Importance of Vector Quantization in Audio Signals Processing)
https://www.dsprelated.com/thread/3543/importance-of-vector-quantization-in-audio-signals-processing
The mathematical construction of GMMs allows people to apply fancy training criterion, especially those that make statistical sense, e.g. max likelihood or max a posteriori or minimum phoneme error rate. That also brings in a lot of computational complexity though. On the contrary, plain VQ requires just a discrete HMM, trainable with the textbook version of EM algorithm, an order of magnitude faster than HMM-GMM.

By 2000s computers are fast enough that none of those presents a problem unless you're doing the so called deep learning stuffs.

## DUDAS PARA XAVIER:
* Fuzzy clustering para codebook: En vez de fuzzy clustering, podriamos hacer el codebook con "hard" clustering (eg k-means) y hacer k-nn <1? (entonces quiza no sabríamos el porcentaje?)